In [1]:
import pandas as pd
import psycopg2
db_username="Jay"
db_pass="Mehrad1"
db_host="127.0.0.1"
db_port="5432"
db_database="FinWisev12"

In [2]:
def Insert_PUBLISHERS(df):
    try:

        connection = psycopg2.connect(user=db_username,
                                      password=db_pass,
                                      host=db_host,
                                      port=db_port,
                                      database=db_database)
        cursor = connection.cursor()
        postgres_insert_query_cheif = """
         DO 
            $$
            BEGIN
                IF NOT EXISTS (select from   public."Publishers" where "persianName"=%(Name)s ) THEN
                   INSERT INTO public."Publishers"(
                    "persianName", "FullName", "TypeOfCompany", "Status", "Industry", "Url")
                    VALUES (%(Name)s, %(FullName)s, %(Type)s, %(Acc)s, %(Industry)s, %(URL)s);
                END IF;
            END
            $$ 

        """
        cursor.executemany(postgres_insert_query_cheif,df.to_dict(orient='records'))
        #######
        
        connection.commit()
        print('PUBLISHERS DONE')
    except(Exception, psycopg2.Error) as error:
            if(connection):
                print('Failed to Update PUBLISHERS', error)
    finally:
        if(connection):
            cursor.close()
            connection.close()  

In [3]:
DF=pd.read_csv('PUBLISHERSTOCK.csv',header=None)
DF.columns=['Pub','Stock']

In [4]:
DF2=pd.read_csv('STOCK.csv',header=None)
DF2=DF2[[0,1]]
DF2.columns=['Stock','Ticker']

In [5]:
DF3=pd.read_csv('PUBLISHERS.csv',header=None)
DF3.columns=['Pub','Name','FullName','Type','Acc','Industry','URL']
DF3=DF3[DF3['Name'].notna()]
Insert_PUBLISHERS(DF3)
DF4=DF3[['Pub','Name']]

PUBLISHERS DONE


In [6]:
DF4

,Pub,Name
0,1,آبادا
1,2,آبان بازار
2,3,آبفا اصفهان
3,4,آبفامشهد
4,5,آب و فاضلاب استان آذربایجان شرقی
...,...,...
2216,2347,وبازار
2217,2348,ویستا
2218,2349,ویسرو
2219,2350,تامین سرمایه آرمان


In [7]:
DF

,Pub,Stock
0,35,965
1,1,1
2,6,1101
3,7,2
4,68,4
...,...,...
726,2002,702
727,2005,1200
728,2215,1200
729,2006,1034


In [8]:
X=pd.merge(DF,DF4)

In [9]:
Y=X.merge(DF2)

In [10]:
Y

,Pub,Stock,Name,Ticker
0,35,965,آ س پ,آ س پ
1,1,1,آبادا,آبادا
2,6,1101,آبین,آبین
3,7,2,آپ,آپ
4,68,4,اپال,اپال
...,...,...,...,...
726,2002,702,ونیکی,ونیکی
727,2005,1200,وهنر,وهنر
728,2215,1200,وهنر,وهنر
729,2006,1034,وهور,وهور


In [11]:
def Insert_PS(df):
    try:

        connection = psycopg2.connect(user=db_username,
                                      password=db_pass,
                                      host=db_host,
                                      port=db_port,
                                      database=db_database)
        cursor = connection.cursor()
        postgres_insert_query_cheif = """
         DO 
            $$
            BEGIN
                IF NOT EXISTS (select from    public."PublisherStock" where "PublisherID"=(SELECT "ID" from public."Publishers" where "persianName"=%(Name)s )) THEN
                INSERT INTO public."PublisherStock"(
                "PublisherID", "StockID")
                VALUES ((SELECT "ID" from public."Publishers" where "persianName"=%(Name)s ), (SELECT "ID" from public."Stocks" where "ticker"=%(Ticker)s ));
                END IF;
            END
            $$ 

        """
        cursor.executemany(postgres_insert_query_cheif,df.to_dict(orient='records'))
        #######
        
        connection.commit()
        print('PS DONE')
    except(Exception, psycopg2.Error) as error:
            if(connection):
                print('Failed to Update PS', error)
    finally:
        if(connection):
            cursor.close()
            connection.close()  

In [16]:
Insert_PS(Y)

PS DONE


In [15]:
Y=Y[Y['Ticker']!='مدار']